<h1>Fine tuning LM for QA<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读取" data-toc-modified-id="读取-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读取</a></span><ul class="toc-item"><li><span><a href="#分词器" data-toc-modified-id="分词器-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>分词器</a></span></li><li><span><a href="#find-top-K-sents" data-toc-modified-id="find-top-K-sents-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>find top K sents</a></span></li><li><span><a href="#测试find-span" data-toc-modified-id="测试find-span-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>测试find span</a></span></li><li><span><a href="#计算span" data-toc-modified-id="计算span-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>计算span</a></span></li></ul></li><li><span><a href="#初始化" data-toc-modified-id="初始化-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>初始化</a></span></li><li><span><a href="#QA-dataset" data-toc-modified-id="QA-dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>QA dataset</a></span></li><li><span><a href="#LSTM-GRU" data-toc-modified-id="LSTM-GRU-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>LSTM GRU</a></span></li><li><span><a href="#排列组合" data-toc-modified-id="排列组合-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>排列组合</a></span></li></ul></div>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from transformers import AutoConfig, AutoModel, AutoTokenizer
from apex import amp

from classes import *
from GNN import *

# 读取

In [7]:
# Restore
classifier = GAT_HotpotQA(features=768, hidden=32, nclass=2, dropout=0.2, alpha=0.3, nheads=8, nodes_num=450)
classifier.to('cuda')
optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()),
                      lr=1e-3)
checkpoint = torch.load('save_cache/GNN_HotpotQA.pt')

opt_level = 'O1'
classifier, optimizer = amp.initialize(classifier, optimizer, opt_level=opt_level)

classifier.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
amp.load_state_dict(checkpoint['amp'])

# Continue training

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [9]:
dataset = HotpotQA_Dataset_New.build_dataset(i_from = 0,i_to = 10)
dataset.set_parameters(450, 0)
print(dataset)


HotpotQA Dataset. mode: train. size: 7. max_seq: 450


## 分词器

```python
classtransformers.XLNetTokenizer(vocab_file, do_lower_case=False, remove_space=True, keep_accents=False, bos_token='<s>', eos_token='</s>', unk_token='<unk>', sep_token='<sep>', pad_token='<pad>', cls_token='<cls>', mask_token='<mask>', additional_special_tokens=['<eop>', '<eod>'], **kwargs)

batch_encode_plus(batch_text_or_text_pairs: Union[str, List[str]], add_special_tokens: bool = True, max_length: Optional[int] = None, stride: int = 0, truncation_strategy: str = 'longest_first', pad_to_max_length: bool = False, return_tensors: Optional[str] = None, return_token_type_ids: Optional[bool] = None, return_attention_masks: Optional[bool] = None, return_overflowing_tokens: bool = False, return_special_tokens_masks: bool = False, return_offsets_mapping: bool = False, return_input_lengths: bool = False, **kwargs)
```

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer


In [2]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

model_name = 'xlnet-base-cased'

proxies={"http_proxy": "127.0.0.1:10809",
         "https_proxy": "127.0.0.1:10809"}
proxies=None

tokenizer_XLNET = AutoTokenizer.from_pretrained(model_name,proxies=proxies)

In [3]:
tokenizer_XLNET.encode('what time is it? it is 9 o\'clock.')

[113, 92, 27, 36, 82, 36, 27, 664, 17, 155, 26, 12431, 9, 4, 3]

In [38]:
ii = [i for i in range(0,18)]
print(tokenizer_XLNET.decode(ii))

<unk><s></s><cls><sep><pad><mask><eod><eop>.()"-–£€


In [48]:
tokenizer_XLNET.encode(('how are you'),add_special_tokens=False)

[160, 41, 44]

In [46]:
tokenizer_XLNET.batch_encode_plus([('what time is it?', 'it is 9 o\'clock.'),
                       ('how are you?', 'i am fine')], max_length = 20, pad_to_max_length=True, return_tensors='pt')

{'input_ids': tensor([[    5,     5,     5,     5,   113,    92,    27,    36,    82,     4,
             36,    27,   664,    17,   155,    26, 12431,     9,     4,     3],
         [    5,     5,     5,     5,     5,     5,     5,     5,     5,   160,
             41,    44,    82,     4,    17,   150,   569,  1592,     4,     3]]),
 'token_type_ids': tensor([[3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
         [3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2]]),
 'attention_mask': tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
tokenizer.decode([   17, 11368,    19,    94,  2288,    27, 10920,     4,     3])

'Hello, my dog is cute<sep><cls>'

In [ ]:
batch_dict_for_LM['input_ids']
batch_dict_for_LM['token_type_ids']
batch_dict_for_LM['attention_mask']
batch_dict_for_LM['start_positions']
batch_dict_for_LM['end_positions']
batch_dict_for_LM['is_impossible']

## find top K sents

In [57]:
for batch_dict in gen_batches(dataset,
                                batch_size=7, 
                                device='cuda'):
    break

In [58]:
batch_dict.keys()

dict_keys(['feature_matrix', 'adj', 'sent_mask', 'para_mask', 'labels', 'answer_type', 'ans_yes_no', 'ques_tokens', 'answer_tokens', 'sent_tokens'])

In [ ]:
def find_ans_spans(target, tokens, offets_type = 'position', top_num = None):
    assert offets_type in ['position', 'range'] and \
        type(target) == type(tokens) == list
    len_x1 = len(target)
    len_x2 = len(tokens)
    if len_x1 == 0 or len_x2 == 0 or len_x1 > len_x2:
        return [0,0]
    
    i1=0
    i2=0
    i2_current = 0
    spans = []
    while i2 <= len_x2 - len_x1:
        if top_num and len(spans) == top_num: # early stop
            break
        i2_current = i2
        while i1 < len_x1:
            if target[i1] != tokens[i2]: 
                i1 = 0
                i2 = i2_current + 1
                break
            else:
                i1 += 1
                i2 += 1
                
        if not i1 < len_x1:
            i1 = 0
            if offets_type == 'position':  
                spans.append((i2_current, i2_current+len_x1-1))
            else:
                spans.append((i2_current, i2_current+len_x1))
                
    return spans[:top_num+1] if top_num else spans[:]

In [ ]:
def convert_batch_to_LM(classifier, batch_dict, topN = 4, 
                        max_length = 20, device = 'cpu'):
    '''convert batch from GNN to LM model(QA fine-tuning)'''
    batch_dict_for_LM = {}
    
    logits_sent, _, logits_Qtype = classifier(batch_dict['feature_matrix'], batch_dict['adj'])
    # logits_Qtype: [B, 2]  1:yesno, 0:span
    _, predict_Qtype = torch.max(logits_Qtype, dim=-1).tolist()
    topN_sents = []
    
    # choose top 4 sents
    for batch_index in range(batch_dict['sent_mask'].shape[0]):
        sent_index = torch.nonzero(batch_dict['sent_mask'][batch_index].view(-1,).long()).flatten()
        sent_logits = torch.index_select(logits_sent[batch_index], 0, sent_index)
        sent_logits_soft = F.softmax(sent_logits, dim = -1).cpu()
        value, _index = torch.max(sent_logits_soft, dim=-1)
        sent_scores = value.tolist()
        sent_score_index = [(score, index) for score, index in zip(sent_scores, range(len(sent_scores)))]
        sent_score_index_sort = sorted(sent_score_index,key=lambda x:x[0], reverse=True)
        sent_indexes = [x[-1] for x in sent_score_index_sort[:topN]]
        topN_sents.append(sent_indexes)
        
    batch_sent_tokens = []
    for batch_index, sents in enumerate(topN_sents):
        temp = []
        for i_sent in sents: temp.extend(batch_dict['sent_tokens'][batch_index][i_sent])
        batch_sent_tokens.append(temp)

    batch_ques_sent = [(batch_dict['ques_tokens'],sent_tokens) for sent_tokens in batch_sent_tokens]
    
    # build start-end positions
    ans_idx = tokenizer_XLNET.encode(batch_dict['answer_tokens'], add_special_tokens=False)
    start_end_list = []
    
    res = tokenizer_XLNET.batch_encode_plus(batch_ques_sent, 
                                            max_length = max_length, 
                                            pad_to_max_length=True, return_tensors='pt')
    
    batch_dict_for_LM['input_ids'] = res['input_ids']
    batch_dict_for_LM['token_type_ids'] = res['token_type_ids']
    batch_dict_for_LM['attention_mask'] = res['attention_mask']
    
    bsz = batch_dict_for_LM['input_ids'].shape[0]
    tokens = batch_dict_for_LM['input_ids'].tolist()
    start_end_list = [find_ans_spans(ans_idx, input_ids, top_num = 1) \
                      for i in range(bsz) \
                      for input_ids in tokens[i]]
    
#     is_impossible = []
    start_positions, end_positions = [], []
    for s_e in start_end_list:
        if s_e == []: 
#             is_impossible.append(1)
            start_positions.append(9e9)
            end_positions.append(9e9)
        else:
#             is_impossible.append(0)
            start_positions.append(s_e[0])
            end_positions.append(s_e[1])          

#     batch_dict_for_LM['is_impossible'] = torch.tensor(is_impossible)
    batch_dict_for_LM['start_positions'] = torch.tensor(start_positions)
    batch_dict_for_LM['end_positions'] = torch.tensor(end_positions)

    for k in batch_dict_for_LM.keys():
        batch_dict_for_LM[k] = batch_dict_for_LM[k].to(device)

    
    

topN_sents_one_batch(batch_dict,logits_sent)

In [17]:
sent_index = torch.nonzero(batch_dict['sent_mask'][0].view(-1,).long()).flatten()

In [18]:
sent_index.to('cpu')

tensor([  2,  10,  14,  22,  31,  33,  36,  43,  49,  54,  58,  66,  72,  74,
         77,  79,  81,  85,  89,  93,  97, 101, 105, 106, 108, 112, 117, 121,
        123, 127, 130, 134, 138, 143, 148, 151, 157])

In [19]:
sent_index.shape

torch.Size([37])

In [20]:
len(batch_dict['sent_tokens'][0])

37

In [21]:
sent_logits = torch.index_select(logits_sent[0], 0, sent_index)
sent_logits_soft = F.softmax(sent_logits, dim = -1).cpu()
sent_logits_soft.shape

torch.Size([37, 2])

In [22]:
value, _index = torch.max(sent_logits_soft, dim=-1)
sent_scores = value.cpu().tolist()
len(sent_scores)

37

In [23]:
sent_score_index = [(score, index) for score, index in zip(sent_scores, range(len(sent_scores)))]

In [24]:
sent_score_index_sort = sorted(sent_score_index,key=lambda x:x[0],reverse=True)
sent_indexes = [x[-1] for x in sent_score_index_sort[:4]]
sent_indexes
# 获取batch中一个元素的前4句.

[1, 14, 12, 16]

## 测试find span

In [66]:
sent_tokens = batch_dict['sent_tokens']

sent_final_tokens = []
for i in sent_indexes: sent_final_tokens.extend(sent_tokens[0][i])
print(sent_final_tokens)

['▁It', '▁was', '▁founded', '▁in', '▁Pine', '▁Bluff', ',', '▁Arkansas', '▁in', '▁1986', '.', '▁Cardinal', '▁Health', '▁also', '▁owns', '▁the', '▁franchise', '▁chain', '▁The', '▁Medicine', '▁Shop', 'pe', '.', '▁Leader', '▁Drug', '▁Store', 's', '▁is', '▁a', '▁network', '▁of', '▁over', '▁3', ',', '100', '▁independently', '▁owned', '▁and', '▁operated', '▁pharmacies', '.', '▁Gray', '▁Drug', '▁was', '▁a', '▁drug', 'store', '▁chain', '▁in', '▁Cleveland', ',', '▁Ohio', '.']


In [67]:
answer_tokens_test =  ['▁network', '▁of', '▁over']

In [68]:
answer_tokens_test in sent_final_tokens

False

In [82]:
print(answer_tokens_test)
print(sent_final_tokens[29:32])

['▁network', '▁of', '▁over']
['▁network', '▁of']


In [15]:
def find_ans_spans(target, tokens, offets_type = 'position', top_num = None):
    assert offets_type in ['position', 'range'] and \
        type(target) == type(tokens) == list
    len_x1 = len(target)
    len_x2 = len(tokens)
    if len_x1 == 0 or len_x2 == 0 or len_x1 > len_x2:
        return [0,0]
    
    i1=0
    i2=0
    i2_current = 0
    spans = []
    while i2 <= len_x2 - len_x1:
        if top_num and len(spans) == top_num:
            break
        i2_current = i2
        while i1 < len_x1:
            if target[i1] != tokens[i2]: 
                i1 = 0
                i2 = i2_current + 1
                break
            else:
                i1 += 1
                i2 += 1
                
        if not i1 < len_x1:
            i1 = 0
            if offets_type == 'position':  
                spans.append([i2_current, i2_current+len_x1-1])
            else:
                spans.append([i2_current, i2_current+len_x1])
                
    return spans[:top_num+1] if top_num else spans[:]

In [87]:
find_ans_spans(answer_tokens_test, sent_final_tokens)

[[29, 31]]

## 计算span

将`ques_feature`和`content_features`加上`[SEP] [CLS]`拼接在一起.

In [88]:
answer_tokens_test2 = [ 'Australian', 'GT', 'Championship',]

In [89]:
sent_tokens_combine = []
for s_index in sent_indexes: sent_tokens_combine.extend(sent_tokens[0][s_index])

In [90]:
final_tokens = ['<x1>','[ques]','<x2>'] + sent_tokens_combine + ['<cls>']
ans_spans = find_ans_spans(answer_tokens_test2, final_tokens, top_num = 2)
ans_spans

[]

In [76]:
len(final_tokens)

57

# 初始化


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import ujson as json
import re
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

json_train_path = r'./data/hotpot_train_v1.1.json'

In [2]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

model_path = '/g/data/models/xlnet-base-cased'
tokenizer_XLNET = AutoTokenizer.from_pretrained(model_path)

In [35]:
tokenizer_XLNET.save_pretrained('/g/data/models/xlnet-base-cased/')

('/g/data/models/xlnet-base-cased/spiece.model',
 '/g/data/models/xlnet-base-cased/special_tokens_map.json',
 '/g/data/models/xlnet-base-cased/added_tokens.json')

In [3]:
with open(json_train_path, 'r', encoding='utf-8') as fp:
    json_train = json.load(fp)
    
json_train[0].keys()

dict_keys(['supporting_facts', 'level', 'question', 'context', 'answer', '_id', 'type'])

In [4]:
item = json_train[0]

In [5]:
len(json_train)

90447

# QA dataset

In [64]:
import os
os.remove?

In [43]:
from datasets import HotpotQA_QA_Dataset, find_ans_spans, generate_QA_batches
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering

In [44]:
tt = HotpotQA_QA_Dataset.build_dataset('data/hotpot_train_mini.json')
# tt = HotpotQA_QA_Dataset.build_dataset('data/hotpot_train_v1.1.json')

In [65]:
tokenizer = AutoTokenizer.from_pretrained('data/models/bert-base-cased')

In [66]:
str(type(tokenizer)).split('.')[-1].split('\'')[0]

'BertTokenizer'

In [67]:
tokenizer.encode('hi i am jim.')

[101, 20844, 178, 1821, 23220, 1306, 119, 102]

In [68]:
tokenizer.decode([i for i in range(10)])

'[PAD] [unused1] [unused2] [unused3] [unused4] [unused5] [unused6] [unused7] [unused8] [unused9]'

In [71]:
tokenizer.batch_encode_plus([('hello i am jim', 'how are you. i am fine thx')],
                           return_special_tokens_masks=True,
                           pad_to_max_length=True,
                           max_length=30,
                           return_token_type_ids=True,
                           return_tensors='pt')

{'special_tokens_mask': tensor([[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101, 19082,   178,  1821, 23220,  1306,   102,  1293,  1132,  1128,
            119,   178,  1821,  2503, 24438,  1775,   102,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0]])}

In [61]:
tokenizer.cls_token_id

0

In [41]:
tokenizer.eos_token_id

2

In [42]:
tokenizer.sep_token_id

4

In [11]:
tt.set_parameters(tokenizer = tokenizer,topN_sents=4, max_length=512, uncased=True, permutations=False)
tt

HotpotQA QA Dataset. mode: train. size: 350. sents num: 4

In [13]:
tt.check_all()

train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sent: 5
train 46. 5ae0d91e55429924de1b7198 no answer: 2006	support sen

In [15]:
tt.check_supporting_facts('5a904e725542995651fb5118')

supporting_facts: [['Just the Two of Us (Grover Washington Jr. song)', 0], ['Just the Two of Us (Grover Washington Jr. song)', 2], ['Bette Davis Eyes', 0], ['Bette Davis Eyes', 1]]
answer: nine

Just the Two of Us (Grover Washington Jr. song)
"Just the Two of Us" is a 1981 R&B single written by Bill Withers, William Salter and Ralph MacDonald, which was recorded by Grover Washington Jr. and Bill Withers.
Just the Two of Us (Grover Washington Jr. song)
 An edited version reached number two on the "Billboard" Hot 100, staying there for 3 weeks, behind "Morning Train (9 to 5)" by Sheena Easton and "Bette Davis Eyes" by Kim Carnes.
Bette Davis Eyes
"Bette Davis Eyes" is a song written and composed by Donna Weiss and Jackie DeShannon, and made popular by American singer Kim Carnes.
Bette Davis Eyes
 DeShannon recorded it in 1974; Carnes's 1981 version spent nine weeks at No. 1 on the "Billboard" Hot 100 and was "Billboard"' s biggest hit of 1981.


In [12]:
tt.set_split('train')

In [13]:
tt.set_split('train')
print('train')
for i in range(350):
    temp = tt.__getitem__(i)
    if int(temp['start_positions'].item()) == 115:
        print(i)
tt.set_split('val')
print('val')
for i in range(150):
    temp = tt.__getitem__(i)
    if int(temp['start_positions'].item()) == 115:
        print(i)


train
50
val
103


In [15]:
tt.set_split('train')
tt.__getitem__(50 ,detail_mod=True)

emprise bank has branches in a city situated along what river 
[('emprise bank has branches in a city situated along what river ', 'emprise bank emprise bank is a kansas based family owned community bank headquartered in wichita kansas with more than 1 billion in assets  emprise bank emprise has 37 locations across the state of kansas including branches in wichita andover augusta chanute clearwater council grove derby el dorado eureka goddard hays haysville hillsboro humboldt iola lawrence mcpherson moran mulvane park city potwin rose hill and valley center  humboldt kansas humboldt is a city in allen county kansas united states  humboldt kansas it is situated along the neosho river ')]
neosho river
[4368, 9468, 341]
input_remove_quesid
[2, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, 3, 16453, 11864, 965, 16453, 11864, 965, 25, 21, 3831, 432, 190, 1467, 514, 965, 9511, 19, 21666, 3831, 29, 91, 119, 137, 2786, 19, 6223, 16453, 11864, 965, 16453, 11864, 63, 4294, 409

{'input_ids': tensor([[    2, 16453, 11864,   965,    63,  4395,    19,    21,   136,  2692,
            303,    98,   341,     3, 16453, 11864,   965, 16453, 11864,   965,
             25,    21,  3831,   432,   190,  1467,   514,   965,  9511,    19,
          21666,  3831,    29,    91,   119,   137,  2786,    19,  6223, 16453,
          11864,   965, 16453, 11864,    63,  4294,  4095,   464,    14,   146,
             16,  3831,   215,  4395,    19, 21666,    17,  2549, 15774,  6122,
           6567,  1207,  2831,   516,  5695,  6473,   931,   107, 14293, 24081,
          24721,  4301,    18,  4301,  5921,   948, 20101, 21410,    31,  2268,
           4439, 26959, 16378,  6633,  2686,    62,   405,   136,  3997,  4181,
           1092,   948,    17,  1152,   459, 21410,  3831, 21410,    25,    21,
            136,    19,  3675,   271,  3831,   181,   202, 21410,  3831,    32,
             25,  2692,   303,    14,  4368,  9468,   341,     3,     0,     0,
              0,     0,    

In [18]:
len([2, 16453, 11864,   965,    63,  4395,    19,    21,   136,  2692,
            303,    98,   341,     3])


14

In [19]:
len([2, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, -99, 3])

15

In [44]:
temp = tt.__getitem__(3)

In [46]:
print(temp['special_tokens_mask'].view(-1, 512).gather(-1, temp['end_positions'].view(-1, 1)))

tensor([[0]])


In [41]:
qq[117]

0

In [61]:
for batch in generate_QA_batches(tt,32, shuffle=False):
    print(batch['start_positions'])

tensor([  35,   75,   48,   82,   30,   62,   15,   12,   83,   26,   31,   85,
          43, -100,   37,   52,   48,   60,   68,   88,   55,   86,   56,    8,
          49,   81,  186,  198,   84,   60,   59,   44])
tensor([  52,   13, -100,   64,   35,   41,   36,   57,   55,  130,   28,   31,
          35,   47,  138,   79,   60,   55,  115,   53,   27,   53,   23,   78,
         113,   39,   74,   72,   30,   66,   40,   46])
tensor([  49,  147,    6,  103, -100,   67,    6,   59,   26,   43,   48,   99,
        -100,   28,   87,   16,   35,   78,   80,   89,   10,  126,   64,   15,
          54,   78,   71,   63,  128,   59,   60,   73])
tensor([ 72,  12,  59,  33,  43, 103,  70,  56,  19,  51,  74,  35,  39,  29,
         68,  59,  57,  85,  16,  98,  54,  55,  88,  55, 115,  91,  69,  43,
         17,  92,  49,  83])
tensor([ 76,  69,  86, 102,   2,  55,  10,  14,  18,  55,  52,  14, 103,  42,
         43,  23,  44,  82,  27, 116, 126,  39,  46, 119,  24,  80,  43, 110,
        

In [35]:
tt.get_item_err_detail_by_id('adf9e7e5542992d7e9f93be')

In [ ]:
tt.check_all()

5ab4c2925542996a3a969fc5 no answer: south america	support sent: 2
5a8299e255429954d2e2eb76 no answer: benjamin wills newton	support sent: 9
5a7b23ca554299042af8f703 index err.
5a8f81d955429918e830d248 no answer: flowering plant	support sent: 2
5ae7b41c5542997ec27276d5 no answer: play	support sent: 2
5add33c05542992c1e3a2562 no answer: north korea	support sent: 3
5abc573d55429959677d6a8a no answer:  full metal panic rpg 	support sent: 3
5ae2168f554299495565d9f7 no answer:  temperatures rising 	support sent: 8
5a8a317455429930ff3c0cef index err.
5adf8f98554299025d62a2bf no answer: degree	support sent: 5
5abbb29c5542993f40c73b7f no answer: venezuela	support sent: 2
5ab6460b5542995eadeeff96 index err.
5ade637955429939a52fe897 no answer: south africa	support sent: 2
5a8051055542992bc0c4a6f7 no answer:  	support sent: 3
5a865501554299211dda2af6 no answer: howard g kazanjian	support sent: 8
5ae3d5bb5542994393b9e77f no answer: south korea	support sent: 2
5addee6e5542997dc79070c0 no answer: sou

In [37]:
tokenizer.decode([1237])

'unit'

In [20]:
tokenizer.encode('China',add_special_tokens=False)

[8481]

In [14]:
tokenizer.encode('division in Hunan Province, China; it is under the administration of Zhuzhou')

[0,
 2757,
 11,
 17087,
 260,
 16517,
 6,
 436,
 131,
 24,
 16,
 223,
 5,
 942,
 9,
 20045,
 4987,
 24181,
 2]

In [5]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
# target = torch.empty(3, dtype=torch.long).random_(5)
target = torch.tensor([0,1,-100], dtype=torch.long)
print(target)
output = loss(input, target)
output

tensor([[ 0.5179,  0.1814,  1.5070, -0.0687,  1.6602],
        [ 1.2503, -1.4755, -0.1997, -1.9947, -0.7214],
        [-1.1893, -1.5768,  0.7894,  1.2449, -0.5401]], requires_grad=True)
tensor([   0,    1, -100])


tensor(2.6039, grad_fn=<NllLossBackward>)

In [6]:
from XLNET_QA_model import XLNetForQuestionAnswering_customized as XLNetQAModel

model_path = '/g/data/models/xlnet-base-cased'

model = XLNetQAModel.from_pretrained(model_path,local_files_only=True)
_ = model.eval()
_ = model.to('cuda')

In [7]:
def compute_span_accuracy(start_logits, start_positions, end_logits, end_positions):

    _, start_indices = start_logits.max(-1)
    start_indices.eq(start_positions)

    _, end_indices = end_logits.max(-1)
    end_indices.eq(end_positions)

    correct = start_indices.eq(start_positions) * end_indices.eq(end_positions)

    numerator = correct.sum().item()
    denominator = start_positions.ne(-100).sum().item()

    # all questions are yes-no type.
    if denominator == 0: return 0

    return float(numerator) / denominator

def compute_accuracy(logits, labels):

    _, logits_indices = logits.max(-1)    

    numerator = torch.eq(logits_indices, labels).sum().item()
    denominator = labels.ne(-100).sum().item()

    if denominator == 0: return 0
    return float(numerator) / denominator

In [9]:
loss_fct = nn.CrossEntropyLoss(ignore_index=-100)

for batch in generate_QA_batches(tt, 16, device='cuda'):
    yes_no_span = batch.pop('yes_no_span')

    res = model(**batch)
    
    start_logits, end_logits, cls_logits = res[0], res[1], res[2]
    

    start_loss = loss_fct(start_logits, batch['start_positions'])
    end_loss = loss_fct(end_logits, batch['end_positions'])
    
    start_end_loss = (start_loss + end_loss) / 2
    cls_loss = loss_fct(cls_logits, yes_no_span) * 0.5

    span_accuracy = compute_span_accuracy(start_logits, batch['start_positions'],
                                                end_logits, batch['end_positions'])
    break

In [ ]:
for k,v in batch.items():
    print(k)
    print(v.shape)

In [ ]:
batch['end_positions']

In [ ]:
batch['yes_no_span']

In [ ]:
for i in res:
    print(i.shape)

In [ ]:
res

In [ ]:
start_logits = torch.tensor([
    [1,6,3,4,5],
    [1,2,5,4,2]
])
end_logits = torch.tensor([
    [1,2,3,4,5],
    [1,2,5,4,6]
])

start_positions = torch.tensor([3,4])
end_positions = torch.tensor([4,4])

In [ ]:
def compute_accuracy(logits, labels):
    
    _, logits_indices = logits.max(-1)    
    
    numerator = torch.eq(logits_indices, labels).sum().item()
    denominator = labels.ne(-100).sum().item()
    
    if denominator == 0: return 0
    return float(numerator) / denominator

In [ ]:
x=torch.randn([3,2])

In [ ]:
labels = torch.tensor([-100,0,0,0,1])
labels = labels.index_select(dim=-1, index=labels.ne(-100).long())
labels

In [ ]:
labels.ne(-100)

In [ ]:
model.training

# LSTM GRU 

In [13]:
import torch
import torch.nn as nn

In [16]:
input = torch.randn(3, 5, 7)
h0 = torch.randn(2*2, 3, 7)
c0 = torch.randn(2*2, 3, 7)
rnn = nn.LSTM(input_size=input.shape[-1], hidden_size=7, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
output, (hn, cn) = rnn(input, (h0, c0))

In [17]:
output.shape

torch.Size([3, 5, 14])

In [18]:
output, (hn, cn) = rnn(input)

In [21]:
input = torch.randn(3, 5, 7)

rnn = nn.GRU(input_size=input.shape[-1], hidden_size=7, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
output, hn = rnn(input)

In [22]:
output.shape

torch.Size([3, 5, 14])

In [27]:
from transformers import AutoTokenizer, AutoModel
pretrained_model_path = 'data/models/xlnet-base-cased'
from QA_models import AutoQuestionAnswering

In [29]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path, local_files_only=True)
model = AutoQuestionAnswering.from_pretrained(pretrained_model_path)

In [30]:
for i in model.state_dict().keys():
    print(i)

transformer.mask_emb
transformer.word_embedding.weight
transformer.layer.0.rel_attn.q
transformer.layer.0.rel_attn.k
transformer.layer.0.rel_attn.v
transformer.layer.0.rel_attn.o
transformer.layer.0.rel_attn.r
transformer.layer.0.rel_attn.r_r_bias
transformer.layer.0.rel_attn.r_s_bias
transformer.layer.0.rel_attn.r_w_bias
transformer.layer.0.rel_attn.seg_embed
transformer.layer.0.rel_attn.layer_norm.weight
transformer.layer.0.rel_attn.layer_norm.bias
transformer.layer.0.ff.layer_norm.weight
transformer.layer.0.ff.layer_norm.bias
transformer.layer.0.ff.layer_1.weight
transformer.layer.0.ff.layer_1.bias
transformer.layer.0.ff.layer_2.weight
transformer.layer.0.ff.layer_2.bias
transformer.layer.1.rel_attn.q
transformer.layer.1.rel_attn.k
transformer.layer.1.rel_attn.v
transformer.layer.1.rel_attn.o
transformer.layer.1.rel_attn.r
transformer.layer.1.rel_attn.r_r_bias
transformer.layer.1.rel_attn.r_s_bias
transformer.layer.1.rel_attn.r_w_bias
transformer.layer.1.rel_attn.seg_embed
transform

In [37]:
xx = tokenizer.batch_encode_plus([
                                ('helolo', 'i am '),
                                ('sss', 'dddd')
                            ], return_tensors='pt',
                            pad_to_max_length=True)

In [38]:
xx.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [39]:
model(**xx)[0].shape

torch.Size([2, 12, 768])

In [14]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)
out = transformer_encoder(src)

In [16]:
out.shape

torch.Size([10, 32, 512])

In [19]:
nn.functional.leaky_relu?

# 排列组合

In [13]:
from itertools import permutations
import numpy as np

In [29]:
x = ['hhh', 'aaa', 'sss', 'ffff']
p = [' '.join(i) for i in permutations(x[:3],3)]
np.random.shuffle(p)
p

['sss hhh aaa',
 'hhh aaa sss',
 'hhh sss aaa',
 'aaa hhh sss',
 'sss aaa hhh',
 'aaa sss hhh']

In [24]:
p1 = [' '.join(i) for i in p]
p1

['hhh aaa sss',
 'aaa hhh sss',
 'sss hhh aaa',
 'sss aaa hhh',
 'hhh sss aaa',
 'aaa sss hhh']

In [2]:
import torch

In [4]:
x = torch.randn([3,12])
i = torch.tensor([1,2,3])

In [13]:
x.gather(-1, i.view(-1, 1))

tensor([[ 0.9562],
        [-1.1158],
        [ 0.6481]])

In [22]:
inputs_id = [0,1,1,1,1,1,2,2,5,5,5,5,5]
BOS = 0
EOS = 2
for start in range(len(inputs_id)):
    if inputs_id[start] != BOS: continue
    for end in range(start+1, len(inputs_id)):
        if inputs_id[end] != EOS: continue
        inputs_id[start+1:end] = [-99 for _ in range(end-start)]
        break
    break
inputs_id

[0, -99, -99, -99, -99, -99, -99, 2, 2, 5, 5, 5, 5, 5]